# Solver FJSP con Gurobi
## Flexible Job Shop Problem - Modelo Fattahi et al. 2007

Este notebook implementa un solver para el problema FJSP usando programación lineal entera mixta (MILP) con Gurobi.

In [ ]:
# Librerías necesarias
import gurobipy as gp
from gurobipy import GRB
import json

## Parámetros del modelo

- **J**: conjunto de trabajos
- **M**: conjunto de máquinas  
- **H[j]**: cantidad de operaciones del trabajo j
- **a[j,h,i]**: vale 1 si la máquina i puede hacer la operación h del trabajo j, 0 si no
- **p[j,h,i]**: tiempo que toma hacer la operación h del trabajo j en la máquina i
- **L**: constante Big-M para las restricciones

## Variables de decisión

**Variables binarias:**
- **y[i,j,h]**: vale 1 si la operación h del trabajo j se asigna a la máquina i
- **x[i,j,h,k]**: vale 1 si la operación h del trabajo j ocupa el slot k en la máquina i

**Variables continuas:**
- **t[j,h]**: momento en que empieza la operación h del trabajo j
- **Tm[i,k]**: momento en que la máquina i empieza su slot k
- **Ps[j,h]**: tiempo real de procesamiento de la operación h del trabajo j
- **Cmax**: makespan (tiempo total para terminar todos los trabajos)

## Restricciones principales

1. **Cmax**: debe ser mayor que el fin de todos los trabajos
2. **Tiempo de procesamiento**: se determina según la máquina asignada
3. **Precedencia**: una operación no puede empezar antes de que termine la anterior del mismo trabajo
4. **Secuencia en máquinas**: los slots deben procesarse en orden
5. **Sincronización inferior**: el tiempo del slot no puede ser mayor que el inicio de la operación
6. **Sincronización superior**: el tiempo del slot debe coincidir con el inicio de la operación
7. **Capacidad**: solo asignar operaciones a máquinas capaces
8. **Un solo trabajo por slot**: cada slot procesa máximo una operación
9. **Una máquina por operación**: cada operación se asigna a exactamente una máquina
10. **Vinculación y-x**: si una operación se asigna a una máquina, debe ocupar un slot en ella

## Formato de las instancias Fattahi

Primera línea: `num_trabajos num_maquinas`

Líneas siguientes (una por trabajo):
```
num_ops [num_maquinas_capaces maquina1 tiempo1 maquina2 tiempo2 ...] ...
```

Ejemplo:
```
5 6
3  3 0 147 1 123 2 145  2 3 140 1 130  2 3 150 4 160
```

Significa:
- 5 trabajos, 6 máquinas
- Trabajo 0: 3 operaciones
  - Op 0: puede hacerse en 3 máquinas (M0:147, M1:123, M2:145)
  - Op 1: puede hacerse en 2 máquinas (M3:140, M1:130)
  - Op 2: puede hacerse en 2 máquinas (M3:150, M4:160)

In [ ]:
def leer_instancia_fattahi(archivo):
    """
    Lee un archivo de instancia en formato Fattahi
    Retorna un diccionario con todos los datos del problema
    """
    with open(archivo, 'r') as f:
        lineas = f.readlines()
    
    # Primera línea: cantidad de trabajos y máquinas
    primera = lineas[0].split()
    cant_trabajos = int(primera[0])
    cant_maquinas = int(primera[1])
    
    # Crear conjuntos
    J = list(range(cant_trabajos))
    M = list(range(cant_maquinas))
    
    # Diccionarios para los parámetros
    H = {}  # operaciones por trabajo
    a = {}  # capacidades
    p = {}  # tiempos
    
    # Leer cada trabajo
    idx_linea = 1
    for j in J:
        datos = list(map(int, lineas[idx_linea].split()))
        idx_linea += 1
        
        pos = 0
        H[j] = datos[pos]  # cantidad de operaciones
        pos += 1
        
        # Para cada operación de este trabajo
        for h in range(H[j]):
            cant_maquinas_capaces = datos[pos]
            pos += 1
            
            # Inicializar todo en 0
            for i in M:
                a[(i, j, h)] = 0
                p[(i, j, h)] = 0
            
            # Leer pares (máquina, tiempo)
            for _ in range(cant_maquinas_capaces):
                maquina = datos[pos]
                tiempo = datos[pos + 1]
                pos += 2
                
                a[(maquina, j, h)] = 1
                p[(maquina, j, h)] = tiempo
    
    # Calcular slots máximos por máquina
    K = {
        i: sum(a[(i, j, h)] for j in J for h in range(H[j]))
        for i in M
    }
    
    # Calcular Big-M
    BIGM = sum(
        max(p[(i, j, h)] for i in M if a[(i, j, h)] == 1)
        for j in J for h in range(H[j])
    )
    
    return {
        'J': J,
        'M': M,
        'H': H,
        'a': a,
        'p': p,
        'K': K,
        'BIGM': BIGM
    }

In [ ]:
def resolver_fjsp(datos, limite_tiempo=3600):
    """
    Construye y resuelve el modelo MILP para FJSP
    """
    # Extraer parámetros
    J = datos['J']
    M = datos['M']
    H = datos['H']
    a = datos['a']
    p = datos['p']
    K = datos['K']
    BIGM = datos['BIGM']
    
    # Crear modelo
    modelo = gp.Model("FJSP_Solver")
    modelo.Params.TimeLimit = limite_tiempo
    
    # Variables de decisión
    y = modelo.addVars(
        [(i, j, h) for i in M for j in J for h in range(H[j])],
        vtype=GRB.BINARY,
        name="asignacion"
    )
    
    x = modelo.addVars(
        [(i, j, h, k) for i in M for j in J for h in range(H[j]) for k in range(K[i])],
        vtype=GRB.BINARY,
        name="slot"
    )
    
    t = modelo.addVars(
        [(j, h) for j in J for h in range(H[j])],
        lb=0.0,
        name="inicio_op"
    )
    
    Tm = modelo.addVars(
        [(i, k) for i in M for k in range(K[i])],
        lb=0.0,
        name="inicio_slot"
    )
    
    Ps = modelo.addVars(
        [(j, h) for j in J for h in range(H[j])],
        lb=0.0,
        name="tiempo_proceso"
    )
    
    Cmax = modelo.addVar(lb=0.0, name="makespan")
    
    # Función objetivo: minimizar makespan
    modelo.setObjective(Cmax, GRB.MINIMIZE)
    
    # Restricción 1: Cmax debe ser mayor que el fin de todos los trabajos
    for j in J:
        ultima_op = H[j] - 1
        modelo.addConstr(
            Cmax >= t[j, ultima_op] + Ps[j, ultima_op],
            name=f"makespan_{j}"
        )
    
    # Restricciones por operación
    for j in J:
        for h in range(H[j]):
            # Restricción 2: tiempo de proceso según máquina asignada
            modelo.addConstr(
                Ps[j, h] == gp.quicksum(p[(i, j, h)] * y[i, j, h] for i in M),
                name=f"tiempo_{j}_{h}"
            )
            
            # Restricción 9: cada operación se asigna a una sola máquina
            modelo.addConstr(
                gp.quicksum(y[i, j, h] for i in M) == 1,
                name=f"una_maquina_{j}_{h}"
            )
        
        # Restricción 3: precedencia entre operaciones del mismo trabajo
        for h in range(H[j] - 1):
            modelo.addConstr(
                t[j, h] + Ps[j, h] <= t[j, h + 1],
                name=f"orden_{j}_{h}"
            )
    
    # Restricciones por máquina
    for i in M:
        for j in J:
            for h in range(H[j]):
                # Restricción 7: solo asignar si la máquina puede hacerlo
                modelo.addConstr(
                    y[i, j, h] <= a[(i, j, h)],
                    name=f"capacidad_{i}_{j}_{h}"
                )
                
                # Restricción 10: vincular asignación con slots
                modelo.addConstr(
                    gp.quicksum(x[i, j, h, k] for k in range(K[i])) == y[i, j, h],
                    name=f"vinculo_{i}_{j}_{h}"
                )
                
                # Restricción 4: slots secuenciales
                for k in range(K[i] - 1):
                    modelo.addConstr(
                        Tm[i, k] + p[i, j, h] * x[i, j, h, k] <= Tm[i, k + 1],
                        name=f"secuencia_{i}_{j}_{h}_{k}"
                    )
                
                # Restricciones 5 y 6: sincronizar slots con operaciones
                for k in range(K[i]):
                    modelo.addConstr(
                        Tm[i, k] <= t[j, h] + (1 - x[i, j, h, k]) * BIGM,
                        name=f"sync_bajo_{i}_{j}_{h}_{k}"
                    )
                    
                    modelo.addConstr(
                        Tm[i, k] + (1 - x[i, j, h, k]) * BIGM >= t[j, h],
                        name=f"sync_alto_{i}_{j}_{h}_{k}"
                    )
        
        # Restricción 8: máximo una operación por slot
        for k in range(K[i]):
            modelo.addConstr(
                gp.quicksum(x[i, j, h, k] for j in J for h in range(H[j])) <= 1,
                name=f"slot_unico_{i}_{k}"
            )
    
    # Resolver
    modelo.optimize()
    
    # Mostrar resultados
    if modelo.Status in (GRB.OPTIMAL, GRB.TIME_LIMIT) and modelo.SolCount > 0:
        print(f"\nMakespan encontrado: {Cmax.X:.2f}")
        print(f"Estado: {'Óptimo' if modelo.Status == GRB.OPTIMAL else 'Tiempo límite alcanzado'}")
        print(f"Tiempo de cómputo: {modelo.Runtime:.2f}s")
        
        if modelo.Status == GRB.TIME_LIMIT:
            print(f"Gap: {modelo.MIPGap * 100:.2f}%")
        
        # Mostrar asignaciones
        print("\nAsignación de operaciones a máquinas:")
        for j in J:
            for h in range(H[j]):
                for i in M:
                    if y[i, j, h].X > 0.5:
                        print(f"  Trabajo {j}, Op {h} -> Máquina {i} (inicia en t={t[j,h].X:.2f})")
    else:
        print(f"No se encontró solución. Estado: {modelo.Status}")
    
    # Empaquetar resultados
    resultados = {
        "trabajos": len(J),
        "maquinas": len(M),
        "variables": modelo.NumVars,
        "restricciones": modelo.NumConstrs,
        "estado": modelo.Status,
        "tiempo_ejecucion": modelo.Runtime,
        "makespan": Cmax.X if modelo.SolCount > 0 else None,
        "gap": modelo.MIPGap if modelo.Status == GRB.TIME_LIMIT and modelo.SolCount > 0 else 0.0
    }
    
    return modelo, resultados

In [ ]:
# Cargar metadatos de las instancias
with open("../instances.json", "r") as f:
    instancias = json.load(f)

def obtener_info_instancia(nombre, lista_instancias):
    """Busca la info de una instancia por nombre"""
    for inst in lista_instancias:
        if inst["name"] == nombre:
            return inst
    raise ValueError(f"No se encontró la instancia '{nombre}'")

## Ejecutar una instancia

Vamos a resolver una instancia de ejemplo

In [ ]:
# Elegir instancia
nombre_instancia = "sfjs01"
info = obtener_info_instancia(nombre_instancia, instancias)
ruta_archivo = "../" + info["path"]

# Leer datos
datos = leer_instancia_fattahi(ruta_archivo)

# Resolver
print(f"=== Resolviendo instancia: {nombre_instancia} ===")
modelo, resultado = resolver_fjsp(datos, limite_tiempo=3600)

print("\n=== Resumen ===")
print(f"Trabajos: {resultado['trabajos']}")
print(f"Máquinas: {resultado['maquinas']}")
print(f"Variables: {resultado['variables']}")
print(f"Restricciones: {resultado['restricciones']}")
print(f"Makespan: {resultado['makespan']:.2f}")
print(f"Tiempo: {resultado['tiempo_ejecucion']:.2f}s")

## Calcular gaps de calidad

Comparamos el makespan obtenido con el óptimo conocido (si existe) o con las cotas

In [ ]:
# Calcular gap
makespan_solver = resultado['makespan']

if makespan_solver is not None:
    if info.get("optimum") is not None:
        # Comparar con óptimo conocido
        optimo = info["optimum"]
        gap_opt = (makespan_solver - optimo) / optimo * 100
        
        print(f"\nÓptimo conocido: {optimo}")
        print(f"Makespan obtenido: {makespan_solver:.2f}")
        print(f"Gap respecto al óptimo: {gap_opt:.2f}%")
    
    elif "bounds" in info:
        # Comparar con cotas
        cota_inferior = info["bounds"]["lower"]
        cota_superior = info["bounds"]["upper"]
        
        gap_lb = (makespan_solver - cota_inferior) / makespan_solver * 100
        gap_ub = (cota_superior - makespan_solver) / cota_superior * 100
        
        print(f"\nCota inferior: {cota_inferior}")
        print(f"Cota superior: {cota_superior}")
        print(f"Makespan obtenido: {makespan_solver:.2f}")
        print(f"Gap vs cota inferior: {gap_lb:.2f}%")
        print(f"Gap vs cota superior: {gap_ub:.2f}%")
    
    else:
        print("\nNo hay óptimo ni cotas disponibles para esta instancia")
else:
    print("\nNo se pudo calcular el gap (no hay solución)")